In [14]:
import sys
workPath = '/home/jovyan/workspace'
if not workPath in sys.path:
    sys.path.append(workPath)

import numpy as np
import os
import importlib
import subprocess
import shutil

from planetengine.systems import arrhenius

from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

def local_import(filepath):

    print(rank, os.listdir(os.path.dirname(filepath)))

    modname = os.path.basename(filepath)

    spec = importlib.util.spec_from_file_location(
        modname,
        filepath,
        )
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)

    return module

def save_script(script, name, path):
    if rank == 0:
        tweakedPath = os.path.splitext(script)[0] + ".py"
        newPath = os.path.join(path, name + ".py")
        shutil.copyfile(tweakedPath, newPath)

def load_script(name, path):
    scriptPath = os.path.join(path, name) + '.py'
    scriptModule = local_import(
        scriptPath
        )
    return scriptModule

def make_testdir():

    if rank == 0:
        if not workPath in sys.path:
            sys.path.append(workPath)
        delete_testdir()
        os.makedirs(testPath)
    return testPath

def delete_testdir():
    if rank == 0:
        if os.path.isdir(testPath):
            shutil.rmtree(testPath)

testPath = '/home/jovyan/workspace/out/test'

delete_testdir()
make_testdir()

system = arrhenius.build()
system.locals.temperatureField.data[:] = 0.
system.update()

save_script(system.script, 'system', testPath)

systemModule = load_script('system', testPath)

system_load = systemModule.build()
assert np.allclose(
    system_load.locals.temperatureField.data,
    system.locals.temperatureField.data
    )
system.update()

import planetengine
planetengine.quickShow(system.locals.temperatureField)

if rank == 0:
    print("Done!")

Done!


In [ ]:
subprocess.call(
    ['chmod', '', '777', workPath]
    )
subprocess.call(
    ['chmod', '', '777', outPath]
    )
subprocess.call(
    ['chmod', '', '777', testPath]
    )